In [1]:
# !pip install datasets

In [2]:
import torch
from torch.optim import AdamW
from transformers import AutoImageProcessor, AutoModelForImageClassification
from datasets import load_dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
from tqdm.auto import tqdm

In [3]:
# Load dataset in streaming mode
ds = load_dataset("MITLL/LADI-v2-dataset", streaming=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

In [4]:
print(ds)

IterableDatasetDict({
    train: IterableDataset({
        features: ['image', 'bridges_any', 'buildings_any', 'buildings_affected_or_greater', 'buildings_minor_or_greater', 'debris_any', 'flooding_any', 'flooding_structures', 'roads_any', 'roads_damage', 'trees_any', 'trees_damage', 'water_any'],
        n_shards: 40
    })
    validation: IterableDataset({
        features: ['image', 'bridges_any', 'buildings_any', 'buildings_affected_or_greater', 'buildings_minor_or_greater', 'debris_any', 'flooding_any', 'flooding_structures', 'roads_any', 'roads_damage', 'trees_any', 'trees_damage', 'water_any'],
        n_shards: 40
    })
    test: IterableDataset({
        features: ['image', 'bridges_any', 'buildings_any', 'buildings_affected_or_greater', 'buildings_minor_or_greater', 'debris_any', 'flooding_any', 'flooding_structures', 'roads_any', 'roads_damage', 'trees_any', 'trees_damage', 'water_any'],
        n_shards: 40
    })
})


In [5]:
first_example = next(iter(ds['train']))
print(first_example.keys())

dict_keys(['image', 'bridges_any', 'buildings_any', 'buildings_affected_or_greater', 'buildings_minor_or_greater', 'debris_any', 'flooding_any', 'flooding_structures', 'roads_any', 'roads_damage', 'trees_any', 'trees_damage', 'water_any'])


In [6]:
if 'Labels' in first_example:
  print('Label Exist!')
  print(f" First example label: {first_example['Labels']}")
else:
  print('Label does not exist in this dataset')

Label does not exist in this dataset


In [7]:
first_example

{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=1800x1200>,
 'bridges_any': False,
 'buildings_any': False,
 'buildings_affected_or_greater': False,
 'buildings_minor_or_greater': False,
 'debris_any': False,
 'flooding_any': False,
 'flooding_structures': False,
 'roads_any': False,
 'roads_damage': False,
 'trees_any': True,
 'trees_damage': True,
 'water_any': True}

In [8]:
# List out possible labels
possible_labels = [key for key, value in first_example.items() if key != 'image']

print(f"Possible labels: {possible_labels}")

Possible labels: ['bridges_any', 'buildings_any', 'buildings_affected_or_greater', 'buildings_minor_or_greater', 'debris_any', 'flooding_any', 'flooding_structures', 'roads_any', 'roads_damage', 'trees_any', 'trees_damage', 'water_any']


In [9]:
# Iterate through the dataset
for i, example in enumerate(ds['train']):  # Replace 'train' with the correct split
    image = example['image']

    # Check if 'label' exists or access another field
    label = example.get('label', None)  # Safely get 'label', or None if it doesn't exist

    # If no direct 'label', check other features
    if label is None:
        # Multi-label scenario: list of possible binary labels
        label = {key: value for key, value in example.items() if key != 'image'}

    # Print image and label
    print(f"Image {i}: {image}")
    print(f"Label {i}: {label}")

    if i >= 5:  # Stop after 5 examples
        break

Image 0: <PIL.PngImagePlugin.PngImageFile image mode=RGB size=1800x1200 at 0x79C9C958E950>
Label 0: {'bridges_any': False, 'buildings_any': False, 'buildings_affected_or_greater': False, 'buildings_minor_or_greater': False, 'debris_any': False, 'flooding_any': False, 'flooding_structures': False, 'roads_any': False, 'roads_damage': False, 'trees_any': True, 'trees_damage': True, 'water_any': True}
Image 1: <PIL.PngImagePlugin.PngImageFile image mode=RGB size=1800x1200 at 0x79C9C958E7D0>
Label 1: {'bridges_any': False, 'buildings_any': True, 'buildings_affected_or_greater': False, 'buildings_minor_or_greater': False, 'debris_any': False, 'flooding_any': False, 'flooding_structures': False, 'roads_any': True, 'roads_damage': False, 'trees_any': True, 'trees_damage': False, 'water_any': False}
Image 2: <PIL.PngImagePlugin.PngImageFile image mode=RGB size=1800x1196 at 0x79C9C958F1C0>
Label 2: {'bridges_any': False, 'buildings_any': False, 'buildings_affected_or_greater': False, 'buildings_

In [10]:
# Load the processor and model
processor = AutoImageProcessor.from_pretrained("MITLL/LADI-v2-classifier-large")
model = AutoModelForImageClassification.from_pretrained("MITLL/LADI-v2-classifier-large")

In [11]:
# Set the model to train mode
model.train()

Swinv2ForImageClassification(
  (swinv2): Swinv2Model(
    (embeddings): Swinv2Embeddings(
      (patch_embeddings): Swinv2PatchEmbeddings(
        (projection): Conv2d(3, 192, kernel_size=(4, 4), stride=(4, 4))
      )
      (norm): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): Swinv2Encoder(
      (layers): ModuleList(
        (0): Swinv2Stage(
          (blocks): ModuleList(
            (0-1): 2 x Swinv2Layer(
              (attention): Swinv2Attention(
                (self): Swinv2SelfAttention(
                  (continuous_position_bias_mlp): Sequential(
                    (0): Linear(in_features=2, out_features=512, bias=True)
                    (1): ReLU(inplace=True)
                    (2): Linear(in_features=512, out_features=6, bias=False)
                  )
                  (query): Linear(in_features=192, out_features=192, bias=True)
                  (key): Linear(in_features=192, out_featur

In [12]:
# Define optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

In [13]:
# Define label keys
label_keys = ['bridges_any', 'buildings_any', 'buildings_affected_or_greater', 'buildings_minor_or_greater',
              'debris_any', 'flooding_any', 'flooding_structures', 'roads_any', 'roads_damage',
              'trees_any', 'trees_damage', 'water_any']

In [14]:
# Loss function for multilabel classification
loss_func = torch.nn.BCEWithLogitsLoss()

In [15]:
# Define batch size
batch_size = 8

In [16]:
def process_dataset(model, dataset, split_name, optimizer=None, train=False):
    model.train() if train else model.eval()

    batch_images = []
    batch_labels = []
    all_labels = []
    all_predictions = []
    running_loss = 0.0

    for i, example in enumerate(tqdm(dataset[split_name])):
        image = example['image']
        labels = [int(example[key]) for key in label_keys]  # Convert label dictionary to list of 0s and 1s

        batch_images.append(image)
        batch_labels.append(labels)

        # When the batch is full, process it
        if (i + 1) % batch_size == 0:
            inputs = processor(images=batch_images, return_tensors="pt")

            # Convert labels to float32 to match the loss function's expectation
            labels_tensor = torch.tensor(batch_labels, dtype=torch.float32)

            # Forward pass
            outputs = model(**inputs, labels=labels_tensor)
            loss = outputs.loss
            running_loss += loss.item()

            if train:
                # Backward pass and optimization for training
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            # Get predictions (sigmoid to convert logits to probabilities)
            logits = outputs.logits
            predictions = torch.sigmoid(logits).cpu().detach().numpy()
            all_predictions.extend(predictions)
            all_labels.extend(batch_labels)

            # Reset the batch
            batch_images = []
            batch_labels = []

    # Process remaining images in the last batch
    if batch_images:
        inputs = processor(images=batch_images, return_tensors="pt")

        # Convert labels to float32 to match the loss function's expectation
        labels_tensor = torch.tensor(batch_labels, dtype=torch.float32)

        # Forward pass
        outputs = model(**inputs, labels=labels_tensor)
        loss = outputs.loss
        running_loss += loss.item()

        if train:
            # Backward pass and optimization for training
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Get predictions
        logits = outputs.logits
        predictions = torch.sigmoid(logits).cpu().detach().numpy()
        all_predictions.extend(predictions)
        all_labels.extend(batch_labels)

    # Convert to numpy arrays for evaluation
    all_labels = torch.tensor(all_labels).numpy()
    all_predictions = torch.tensor(all_predictions).numpy()

    return running_loss / len(all_predictions), all_labels, all_predictions

In [17]:
# Function to compute evaluation metrics
def compute_metrics(labels, predictions, threshold=0.5):
    # Convert probabilities to binary predictions using threshold
    predictions_bin = (predictions >= threshold).astype(int)

    # Calculate accuracy, precision, recall, f1, and ROC AUC
    accuracy = accuracy_score(labels, predictions_bin)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions_bin, average='macro', zero_division=1)
    roc_auc = roc_auc_score(labels, predictions, average='macro', multi_class='ovr')

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"ROC AUC: {roc_auc:.4f}")

    return accuracy, precision, recall, f1, roc_auc

In [ ]:
# Training loop
num_epochs = 3
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    train_loss, train_labels, train_preds = process_dataset(model, ds, 'train', optimizer, train=True)
    print(f"Training Loss: {train_loss:.4f}")

    # Validate after each epoch
    val_loss, val_labels, val_preds = process_dataset(model, ds, 'validation')
    print(f"Validation Loss: {val_loss:.4f}")

    # Compute metrics for validation
    compute_metrics(val_labels, val_preds)

Epoch 1/3


0it [00:00, ?it/s]